# プロンプトを自動生成してみよう

`PromptCreator` は、目的と背景を入力として、プロンプトを自動生成するプロンプトです。

PromptGenでは、プロンプトのモデルそのものも入出力パラメータとして扱うことができます。これにより、プロンプトを自動生成するプロンプトを作成することができます。

In [1]:
import promptgen as pg

In [2]:
collection = pg.PromptCollection(load_predefined=True)
print(collection)

<PromptCollection>
Number of prompts: 6

Prompts:
- TextCategorizer: (text: str, categories: list) -> (category: str, found: bool)
- TextSummarizer: (text: str) -> (summary: str)
- PromptExampleCreator: (prompt: Prompt, n: int) -> (examples: list)
- PromptCreator: (purpose: str, background: str) -> (prompt: Prompt)
- PromptOptimizer: (original_prompt: Prompt, background: str) -> (optimized_prompt: Prompt)
- PythonCodeGenerator: (task: str) -> (reason: str, code: str)



## プロンプトを生成するプロンプト

`PromptCreator` は、プロンプトを生成するプロンプトです。

入力パラメータは、以下の通りです。

```python
class PromptCreatorInput(InputValue):
    purpose: str
    background: str
```

出力パラメータは、以下の通りです。

```python
class PromptCreatorOutput(OutputValue):
    prompt: Prompt
```

In [3]:
prompt_creator = collection['PromptCreator']
prompt_creator

Prompt: PromptCreator

Create a prompt from the given purpose. Don't create an example with the input purpose. Instead, create an example with a different purpose. Consider background information that is necessary to understand the purpose.

Input Parameters:
    - purpose (str): purpose of the prompt
    - background (str): background of the prompt

Output Parameters:
    - prompt (Prompt): prompt created from the given purpose. Is has 'name', 'description', 'input_parameters', 'output_parameters', 'template', and 'examples'.

プロンプトを文字列にフォーマットすると、以下のようになります。

In [4]:
formatter = pg.KeyValuePromptFormatter()

print(formatter.format_prompt_without_input(prompt=prompt_creator))

You are an AI named "PromptCreator".
Create a prompt from the given purpose. Don't create an example with the input purpose. Instead, create an example with a different purpose. Consider background information that is necessary to understand the purpose.
You should follow 'Template' format. The format is 'key: value'.

Input Parameters:
  - purpose: purpose of the prompt
  - background: background of the prompt

Output Parameters:
  - prompt: prompt created from the given purpose. Is has 'name', 'description', 'input_parameters', 'output_parameters', 'template', and 'examples'.

Template:
Input:
purpose: """purpose of the prompt"""
background: """background of the prompt"""
Output:
prompt: { 'name': 'sample-new prompt',
  'description': 'A sample prompt.',
  'input_parameters': [{'name': 'input1', 'description': 'The first input parameter.'}, {'name': 'input2', 'description': 'The second input parameter.'}],
  'output_parameters': [{'name': 'output1', 'description': 'The first output p

## 大規模言語モデルの準備

今回は、 `gpt-3.5-turbo` を使います。

In [5]:
import openai
import os

from dotenv import load_dotenv
load_dotenv('../../.env')

openai.api_key = os.getenv("OPENAI_API_KEY")
openai.organization = os.getenv("OPENAI_ORG_ID")

def generate_llm_response(prompt: str, model: str) -> str:
    resp = openai.ChatCompletion.create(model=model, messages=[
        {'role': 'user', 'content': prompt}
    ], max_tokens=2048)
    return resp['choices'][0]['message'].get('content', '')

In [6]:
class PromptCreatorInput(pg.InputValue):
    purpose: str
    background: str

class PromptCreatorOutput(pg.OutputValue):
    prompt: pg.Prompt

In [7]:
raw_req = formatter.format_prompt(prompt_creator, PromptCreatorInput(
    purpose='Generate Python code from given task',
    background='style: input: (task: str), output: (reason: str, code: str)')
)
raw_resp = generate_llm_response(raw_req, 'gpt-3.5-turbo')
resp = PromptCreatorOutput.from_dataclass(formatter.parse(prompt_creator, raw_resp))

code_generator = pg.Prompt.from_dict(resp.prompt)
code_generator

Prompt: PythonCodeGenerator2

Generate Python code based on the given task.

Input Parameters:
    - task (str): The task for which the Python code needs to be generated

Output Parameters:
    - reason (str): The reason for the generated Python code
    - code (str): The Python code generated to complete the task

作成したプロンプトを使って、タスクを分解してみましょう。

In [8]:
raw_req = formatter.format_prompt(code_generator, pg.InputValue.from_dict({
    'task': 'read stdin as stream and write stdout the upper-case string'
}))
raw_resp = generate_llm_response(raw_req, 'gpt-3.5-turbo')

print(raw_resp)

reason: """The task requires converting all characters in a string to upper-case and then reading from stdin and writing to stdout"""
code: """import sys
for line in sys.stdin:
    sys.stdout.write(line.upper())"""


In [9]:
output_value = formatter.parse(code_generator, raw_resp)
output_value

OutputValue(reason='The task requires converting all characters in a string to upper-case and then reading from stdin and writing to stdout', code='import sys\nfor line in sys.stdin:\n    sys.stdout.write(line.upper())')

In [10]:
print(output_value['code'])

import sys
for line in sys.stdin:
    sys.stdout.write(line.upper())
